# MyScaleを使用した埋め込み検索

このノートブックでは、データをダウンロードし、埋め込みを作成し、選択したベクトルデータベースを使用してインデックス化と検索を行う簡単なフローを説明します。これは、チャットボット、トピックモデリングなどの本番ユースケースをサポートするために、セキュアな環境で独自のデータと共に埋め込みを保存・検索したい顧客にとって一般的な要件です。

### ベクトルデータベースとは

ベクトルデータベースは、埋め込みベクトルを保存、管理、検索するために作られたデータベースです。非構造化データ（テキスト、音声、動画など）を機械学習モデルで利用するためのベクトルとしてエンコードする埋め込みの使用は、自然言語、画像認識、その他の非構造化データ形式に関わるユースケースでAIの効果が高まったことにより、近年爆発的に増加しています。ベクトルデータベースは、企業がこれらのユースケースを提供し、スケールするための効果的なソリューションとして登場しました。

### なぜベクトルデータベースを使用するのか

ベクトルデータベースにより、企業はこのリポジトリで共有している多くの埋め込みユースケース（質問応答、チャットボット、推薦サービスなど）を、セキュアでスケーラブルな環境で活用できるようになります。多くの顧客が小規模では埋め込みで問題を解決していますが、パフォーマンスとセキュリティが本番環境への移行を阻んでいます。私たちはベクトルデータベースをその解決の重要な要素と考えており、このガイドではテキストデータの埋め込み、ベクトルデータベースへの保存、セマンティック検索での使用の基本を説明します。

### デモフロー
デモフローは以下の通りです：
- **セットアップ**: パッケージをインポートし、必要な変数を設定
- **データの読み込み**: データセットを読み込み、OpenAI埋め込みを使用して埋め込みを作成
- **MyScale**
    - *セットアップ*: MyScale Pythonクライアントを設定。詳細は[こちら](https://docs.myscale.com/en/python-client/)
    - *データのインデックス化*: テーブルを作成し、__content__用にインデックス化
    - *データの検索*: 様々な目標を念頭に置いたサンプルクエリを実行

このノートブックを実行すると、ベクトルデータベースのセットアップと使用方法の基本的な理解が得られ、埋め込みを活用したより複雑なユースケースに進むことができます。

## セットアップ

必要なライブラリをインポートし、使用したい埋め込みモデルを設定します。

In [ ]:
# We'll need to install the MyScale client
!pip install clickhouse-connect

#Install wget to pull zip file
!pip install wget

In [2]:
import openai

from typing import List, Iterator
import pandas as pd
import numpy as np
import os
import wget
from ast import literal_eval

# MyScale's client library for Python
import clickhouse_connect

# I've set this to our new embeddings model, this can be changed to the embedding model of your choice
EMBEDDING_MODEL = "text-embedding-3-small"

# Ignore unclosed SSL socket warnings - optional in case you get these errors
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## データの読み込み

このセクションでは、このセッション以前に準備した埋め込みデータを読み込みます。

In [ ]:
embeddings_url = 'https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip'

# The file is ~700 MB so this will take some time
wget.download(embeddings_url)

In [ ]:
import zipfile
with zipfile.ZipFile("vector_database_wikipedia_articles_embedded.zip","r") as zip_ref:
    zip_ref.extractall("../data")

In [3]:
article_df = pd.read_csv('../data/vector_database_wikipedia_articles_embedded.csv')

In [4]:
article_df.head()

,id,url,title,text,title_vector,content_vector,vector_id
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...,"[0.001009464613161981, -0.020700545981526375, ...","[-0.011253940872848034, -0.013491976074874401,...",0
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...,"[0.0009286514250561595, 0.000820168002974242, ...","[0.0003609954728744924, 0.007262262050062418, ...",1
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...,"[0.003393713850528002, 0.0061537534929811954, ...","[-0.004959689453244209, 0.015772193670272827, ...",2
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...,"[0.0153952119871974, -0.013759135268628597, 0....","[0.024894846603274345, -0.022186409682035446, ...",3
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...,"[0.02224554680287838, -0.02044147066771984, -0...","[0.021524671465158463, 0.018522677943110466, -...",4


In [5]:
# Read vectors from strings back into a list
article_df['title_vector'] = article_df.title_vector.apply(literal_eval)
article_df['content_vector'] = article_df.content_vector.apply(literal_eval)

# Set vector_id to be a string
article_df['vector_id'] = article_df['vector_id'].apply(str)

In [6]:
article_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              25000 non-null  int64 
 1   url             25000 non-null  object
 2   title           25000 non-null  object
 3   text            25000 non-null  object
 4   title_vector    25000 non-null  object
 5   content_vector  25000 non-null  object
 6   vector_id       25000 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.3+ MB


## MyScale
次に検討するベクトルデータベースは[MyScale](https://myscale.com)です。

[MyScale](https://myscale.com)は、Clickhouse上に構築されたデータベースで、ベクトル検索とSQL分析を組み合わせて、高性能で合理化された完全管理型の体験を提供します。構造化データとベクトルデータの両方に対する結合クエリと分析を促進するよう設計されており、すべてのデータ処理に対して包括的なSQLサポートを提供します。

[MyScale Console](https://console.myscale.com)を使用して、2分以内にクラスター上でSQLによるベクトル検索をデプロイし、実行できます。

### MyScaleに接続する

[接続詳細](https://docs.myscale.com/en/cluster-management/)セクションに従って、MyScaleコンソールからクラスターホスト、ユーザー名、パスワード情報を取得し、以下に示すようにクラスターへの接続を作成してください：

In [8]:
# initialize client
client = clickhouse_connect.get_client(host='YOUR_CLUSTER_HOST', port=8443, username='YOUR_USERNAME', password='YOUR_CLUSTER_PASSWORD')

### インデックスデータ

MyScaleに埋め込みデータを格納するための`articles`というSQLテーブルを作成します。このテーブルには、コサイン距離メトリックを使用したベクトルインデックスと、埋め込みの長さに対する制約が含まれます。以下のコードを使用してarticlesテーブルを作成し、データを挿入してください：

In [9]:
# create articles table with vector index
embedding_len=len(article_df['content_vector'][0]) # 1536

client.command(f"""
CREATE TABLE IF NOT EXISTS default.articles
(
    id UInt64,
    url String,
    title String,
    text String,
    content_vector Array(Float32),
    CONSTRAINT cons_vector_len CHECK length(content_vector) = {embedding_len},
    VECTOR INDEX article_content_index content_vector TYPE HNSWFLAT('metric_type=Cosine')
)
ENGINE = MergeTree ORDER BY id
""")

# insert data into the table in batches
from tqdm.auto import tqdm

batch_size = 100
total_records = len(article_df)

# we only need subset of columns
article_df = article_df[['id', 'url', 'title', 'text', 'content_vector']]

# upload data in batches
data = article_df.to_records(index=False).tolist()
column_names = article_df.columns.tolist()

for i in tqdm(range(0, total_records, batch_size)):
    i_end = min(i + batch_size, total_records)
    client.insert("default.articles", data[i:i_end], column_names=column_names)

  0%|          | 0/250 [00:00<?, ?it/s]

ベクターインデックスはバックグラウンドで自動的に構築されるため、検索を実行する前にビルドステータスを確認する必要があります。

In [10]:
# check count of inserted data
print(f"articles count: {client.command('SELECT count(*) FROM default.articles')}")

# check the status of the vector index, make sure vector index is ready with 'Built' status
get_index_status="SELECT status FROM system.vector_indices WHERE name='article_content_index'"
print(f"index build status: {client.command(get_index_status)}")

articles count: 25000
index build status: InProgress


### データの検索

MyScaleにインデックス化された後、ベクトル検索を実行して類似するコンテンツを見つけることができます。まず、OpenAI APIを使用してクエリの埋め込みを生成します。次に、MyScaleを使用してベクトル検索を実行します。

In [11]:
query = "Famous battles in Scottish history"

# creates embedding vector from user query
embed = openai.Embedding.create(
    input=query,
    model="text-embedding-3-small",
)["data"][0]["embedding"]

# query the database to find the top K similar content to the given query
top_k = 10
results = client.query(f"""
SELECT id, url, title, distance(content_vector, {embed}) as dist
FROM default.articles
ORDER BY dist
LIMIT {top_k}
""")

# display results
for i, r in enumerate(results.named_results()):
    print(i+1, r['title'])

1 Battle of Bannockburn
2 Wars of Scottish Independence
3 1651
4 First War of Scottish Independence
5 Robert I of Scotland
6 841
7 1716
8 1314
9 1263
10 William Wallace
